## Example 05 - Container hub

**Example description:** Example of a large container vessel that brings containers to a stock yard, and smaller inland container vessels that ship them further inland.

* [**0. Import required libraries:**](#0.-Import-required-libraries)<br>
* [**1. Define work method:**](#1.-Define-work-method)<br>
   * [**1.1 Sites:**](#1.1-Define-the-project-sites)<br>
   * [**1.2 Equipment:**](#1.2-Define-the-project-equipment)<br>
   * [**1.3 Activities:**](#1.3-Define-the-activity)<br>
* [**2. Run the simulation:**](#2.-Run-the-simulation)<br>
* [**3. Post processing:**](#3.-Postprocessing)<br>

## 0. Import required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model
import digital_twin.plot as plot

In [2]:
# Create simulation environment
simulation_start = datetime.datetime(2019, 1, 1)

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

## 1. Define work method

### 1.1 Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### First create objects with the desired properties

In [3]:
# The generic site class
Site = type('Site', (core.Identifiable, # Give it a name
                     core.Log,          # Allow logging of all discrete events
                     core.Locatable,    # Add coordinates to extract distance information and visualize
                     core.HasContainer, # Add information on the material available at the site
                     core.HasResource), # Add information on serving equipment
            {})                         # The dictionary is empty because the site type is generic

#### Next specify the properties for sites you wish to create

In [4]:
# Information on the origin site
location_from_site = shapely.geometry.Point(0.4958561848972654,51.50427537860759)  # lon, lat

data_from_site = {"env": my_env,                  # The simpy environment defined in the first cel
                  "name": "Origin locatie",       # The name of the site
                  "geometry": location_from_site, # The coordinates of the project site
                  "capacity": 1_000_000,          # The capacity of the site
                  "level": 1_000_000}             # The actual volume of the site

In [5]:
# Information on the intermediate site
location_inter_site = shapely.geometry.Point(4.067560378249368,51.95173269362142)     # lon, lat

data_inter_site = {"env": my_env,                 # The simpy environment defined in the first cel
                "name": "Overslag locatie",       # The name of the site
                "geometry": location_inter_site,     # The coordinates of the project site
                "capacity": 1_000_000,            # The capacity of the site
                "level": 0}                       # The actual volume of the site (empty of course)

In [6]:
# Information on the destination site 
location_to_site = shapely.geometry.Point(5.83042471825503, 51.86058245744734)     # lon, lat

data_to_site = {"env": my_env,                    # The simpy environment defined in the first cel
                "name": "Destination locatie",    # The name of the site
                "geometry": location_to_site,     # The coordinates of the project site
                "capacity": 1_000_000,              # The capacity of the site
                "level": 0}                       # The actual volume of the site (empty of course)

#### Finally create specific instances of the predefined objects with the specified properties

In [7]:
# The two objects used for the simulation
from_site = Site(**data_from_site)
inter_site = Site(**data_inter_site)
to_site   = Site(**data_to_site)

### 1.2 Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

#### First create objects with the desired properties

In [8]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable,              # Give it a name
                                    core.Log,                       # Allow logging of all discrete events
                                    core.ContainerDependentMovable, # A moving container, so capacity and location
                                    core.Processor,                 # Allow for loading and unloading
                                    core.HasResource,               # Add information on serving equipment
                                    core.HasSoil,                   # Add soil object, because it moves soil
                                    core.HasPlume),                 # Initialize spill terms
                                   {})

#### Next specify the properties for vessel(s) you wish to create

In [9]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

def compute_loading(rate):
    return lambda x: x / rate

def compute_unloading(rate):
    return lambda x: x / rate

In [10]:
# TSHD variables
data_container_large = {"env": my_env,                              # The simpy environment 
               "name": "Container large",                           # Name
               "geometry": location_from_site,                      # It starts at the "from site"
               "loading_func": compute_loading(1.5),                # Loading rate
               "unloading_func": compute_unloading(1.5),            # Unloading rate
               "capacity": 20_000,                                  # Capacity of the container vessel - TEU
               "compute_v": compute_v_provider(5, 4.5)}             # Variable speed 

In [11]:
# TSHD variables
data_container_small = {"env": my_env,                              # The simpy environment 
               "name": "Container small",                           # Name
               "geometry": location_from_site,                      # It starts at the "from site"
               "loading_func": compute_loading(1.5),                # Loading rate
               "unloading_func": compute_unloading(1.5),            # Unloading rate
               "capacity": 600,                                     # Capacity of the container vessel - TEU
               "compute_v": compute_v_provider(5, 4.5)}             # Variable speed 

#### Finally create specific instances of the predefined objects with the specified properties

In [12]:
# The simulation object
container_large = TransportProcessingResource(**data_container_large)
container_small = TransportProcessingResource(**data_container_small)

### 1.3 Define the activity

In [13]:
# Continuous simulation, stop after one year
stop_condition = model.TimeCondition(my_env, datetime.datetime(2019, 1, 1), 
                                             datetime.datetime(2020, 1, 1))

In [14]:
# Create activity
activity_1 = model.Activity(env = my_env,                     #    The simpy environment defined in the first cel
                          name = "International transport", # We are moving soil
                          origin = from_site,               # We originate from the from_site
                          destination = inter_site,         # And therefore travel to the to_site
                          loader = container_large,         # The benefit of a TSHD, all steps can be done
                          mover = container_large,          # The benefit of a TSHD, all steps can be done
                          unloader = container_large,       # The benefit of a TSHD, all steps can be done
                          start_condition = None,           # We can start right away and do not stop
                          stop_condition = stop_condition)  # stop after 1 year simulation

In [15]:
# Create activity                      
activity_2 = model.Activity(env = my_env,                     # The simpy environment defined in the first cel
                          name = "Hinterland transport",    # We are moving soil
                          origin = inter_site,              # We originate from the from_site
                          destination = to_site,            # And therefore travel to the to_site
                          loader = container_small,         # The benefit of a TSHD, all steps can be done
                          mover = container_small,          # The benefit of a TSHD, all steps can be done
                          unloader = container_small,       # The benefit of a TSHD, all steps can be done
                          start_condition = None,           # We can start right away and do not stop
                          stop_condition = stop_condition)  # stop after 1 year simulation

### 2. Run the simulation

In [16]:
my_env.run()

print("\n*** Installation of dike finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now - my_env.epoch))))

T=1546297200.00 Start condition is satisfied, International transport transporting from Origin locatie to Overslag locatie started
T=1546297200.00 Start condition is satisfied, Hinterland transport transporting from Overslag locatie to Destination locatie started
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move


Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

...

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing 

### 3. Postprocessing

#### Vessel planning

In [17]:
container_small.__dict__

{'geometry': <shapely.geometry.point.PointAdapter at 0x21a447e6b38>,
 'env': <simpy.core.Environment at 0x21a447c6b00>,
 'sigma_d': 0.015,
 'sigma_o': 0.1,
 'sigma_p': 0.05,
 'f_sett': 0.5,
 'f_trap': 0.01,
 'm_r': 0,
 'soil': {},
 'resource': <simpy.resources.resource.Resource at 0x21a44800cc0>,
 'loading_func': <function __main__.compute_loading.<locals>.<lambda>(x)>,
 'unloading_func': <function __main__.compute_unloading.<locals>.<lambda>(x)>,
 'container': <simpy.resources.container.Container at 0x21a44800c50>,
 'total_requested': 0,
 'v': 1,
 'wgs84': <pyproj.Geod at 0x21a447e0220>,
 'compute_v': <function __main__.compute_v_provider.<locals>.<lambda>(x)>,
 'log': {'Message': ['sailing empty start',
   'sailing empty stop',
   'loading start',
   'loading stop',
   'sailing full start',
   'sailing full stop',
   'unloading start',
   'unloading stop',
   'sailing empty start',
   'sailing empty stop',
   'loading start',
   'loading stop',
   'sailing full start',
   'sailing fu

In [18]:
vessels = [container_large, container_small]

activities = ['loading', 'unloading', 'sailing full', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

#### KML visualisation

In [19]:
plot.vessel_kml(my_env, vessels, stepsize = 1200)

In [20]:
# open the file
if platform.system():
    !start ./vessel_movements.kml
else:
    !start explorer ./vessel_movements.kml